# How to get these notebooks to work

Dataset describe here: http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

git clone git@github.com:franzoni/proton-Hands-on_task_Senior_Data_Scientist.git

mv online_retail_II.xlsx proton-Hands-on_task_Senior_Data_Scientist

# DATA cleaning and preparation
## repeat the choices already made fr EDA and customers analyes, namely:
- remove nulls
- consider only ordinary transactions
- remove cancellations

In [ ]:
import sys, os
import pandas as pd

data_path = os.getcwd()
print('++ Directory of this notebook:')
print('\t%s'%data_path)

excel_with_path  = data_path+'/online_retail_II.xlsx'
pickle_with_path = data_path+'/online_retail_II.pkl'
csv_with_path    = data_path+'/online_retail_II.csv'

print('\n++Full path of the data file:')
print('\tdata_path =%s\n\n'%excel_with_path)

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster using it than opening excel files\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)
    print('-> picke done\n')
    
df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# implement the definition of cancellation in the documentation:
# http://archive.ics.uci.edu/ml/datasets/Online+Retail+II
#      ==> "If this code starts with the letter 'c', it indicates a cancellation.""
def is_cancellation(s):
    if isinstance(s, int):
        #print "ordinary invoice"
        return 0
    elif isinstance(s, unicode):
        #print "unicode string"
        #thse are cancellations
        if s.rfind('C')!=-1:
            return 1
        else:
            # print "Something unexpected"  # found, e.g. A506401
            # print s
            return 2
df['IsCancellation'] = df['Invoice'].map(is_cancellation)     
      


# items with prices above ~1500 are so few that is worth looking at them in detail, and excluding them from the plots
# There'a lot of transactions with price set to 0, which based on the descriptions are 
max_item_price=1400
r             =(0,max_item_price)
def is_ordinary_item(p):
    if abs(p)>max_item_price or p==0:
        return 0
    else:
        return 1
df['IsOrdinaryItem'] = df['Price'].map(is_ordinary_item) 

In [ ]:
df.shape

In [ ]:
df=df.dropna()
df= df.loc[ (df.IsCancellation==0) & (df.IsOrdinaryItem==1) ]
df.shape

In [ ]:
df.head(2)

##  Are there users at high risk of churning by the end of 2011

- https://www.kdnuggets.com/2019/05/churn-prediction-machine-learning.html 
- https://towardsdatascience.com/hands-on-predict-customer-churn-5c2a42806266
- https://www.google.com/search?q=dwarf+on+the+shoulders+of+giants&rlz=1C5CHFA_enCH771CH771&sxsrf=ALeKk034vLLmAHn5V1c0QAON4DZuO1GqVA:1582207255390&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiRrayrpeDnAhXCxaYKHZQVCgcQ_AUoAXoECBIQAw&biw=1418&bih=915#imgrc=oj4GwW0aHYfVYM
- similar exercise resolved here https://towardsdatascience.com/modeling-customer-churn-for-an-e-commerce-business-with-python-874315e688bf
- the repo of the library to be used https://github.com/CamDavidsonPilon/lifetimes
- the reference paper in pdf http://brucehardie.com/papers/018/fader_et_al_mksc_05.pdf http://brucehardie.com/papers/bgnbd_2004-04-20.pdf and the journal which has all aspects of a respectable journals 
- GF TODO: once you've found which customers have churned, make a few hitory plots to prove that indeed they have churned, to show that I am not blindly trusing the package I've downloade


- cancellations: fraction of cancellation by nunber of transactions and by proportion of renenue
-                correlation to CHURNING ? Correlation to country OR type of good purchased ?


- NOT SURE IF USEFUL some stack-overflow like explanations about the Pareto model https://stats.stackexchange.com/questions/251506/is-it-possible-to-understand-pareto-nbd-model-conceptually

## STRATEGY: 
- calibrate modelMID up to mid 2011
- check the model on the last 6 months
- modelMID gives the probability of alive/death at mid 2011
- re-calibrate over the whole period, modelFULL
- modelFULL gives the probability of alive/death at end
- question 1: p(Alive, modelMID) is high, p(Alive, modelFULL) is low
- question 2: p(Alive, modelFULL) is somewhat low

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from lifetimes.utils import *
from lifetimes import BetaGeoFitter,GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix, plot_period_transactions, plot_cumulative_transactions,plot_incremental_transactions
from lifetimes.generate_data import beta_geometric_nbd_model
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive
from lifetimes.fitters import *

In [ ]:
help(summary_data_from_transaction_data)

##   RFM variables and negative/cancellation transactions
- frequency, recency, T
- how do I deal with transactions, in producing the RFM models ?
- for now just exclude them: cancellations are likely to correlate to churning behaviour, however the frequency of positive transactions is what the pareto model uses and expects to predict churning probabilty
- ===> check if there's correlation between churning behaviour and frequency of cancellations ?  

## FOR the moment I exclude cancellations from the type of transactions to compute RFM variables

In [ ]:
columns_forRFM = ['Customer_ID','InvoiceDate','Transaction']
df_forRFM = df[columns_forRFM]
df_forRFM.shape

In [ ]:
df_forRFM.head(3)

## DOC of function _summary__data__from__transaction__data_
summary_data_from_transaction_data(transactions, customer_id_col, datetime_col, monetary_value_col=None, datetime_format=None, observation_period_end=None, freq='D', freq_multiplier=1)

- Return summary data from transactions.
    
-    This transforms a DataFrame of transaction data of the form:
        customer_id, datetime [, monetary_value]
-    to a DataFrame of the form:
        customer_id, frequency, recency, T [, monetary_value]

In [ ]:
# RFM_total is for the total dataset
RFM_total = summary_data_from_transaction_data(df_forRFM, \
                'Customer_ID','InvoiceDate'
                ,monetary_value_col='Transaction',freq='D')
RFM_total.shape

In [ ]:
RFM_total.head(5)

## RFM info

GENERALLY, the first transaction (aka 'the birth' of the customer) is ecluded from the RFM variables

DEFINITIONS
- Recency: time between initial purchase and most recent (last) purchase
- Frequency: number of repeat purchases made by a customer (total purchases — 1)
- Monetary: total spent on purchases

FUNCTION summary_data_from_transaction_data
- it's a rather clever thing: it AGGREGATES into 1 all transactions taking place at the same DAY!

- frequency: # of days in which a customer made a repeat purchase
- T: customer's age in days
- recency: customer's age in days at the time of the most recent purchase
- monetary_value: mean of a customer's purchases, excluding the 1st purchase

In [ ]:
# a basic glance at the content
RFM_total.describe()

# Verifiaction of how summary_data_from_transaction_data
- for a few Customer_Id randomly picked, Frequency makes sense
- the code behaves ok based on the spot-checks, is publicly available https://github.com/CamDavidsonPilon/lifetimes

# SHOULD I RESTRICT MOST OF THE ANALYSIS AND PLOTS TO CUSTOMERS WHO HAVE RETURNED?
- if a customer has haver come back, it's probably dead anyway...
- and all the values at 0 are somewhat disturbing

## DONE, can remove this box at some point

In [ ]:
RFM_total.shape

In [ ]:
# keep the option of toggling this
if False:
    RFM_total = RFM_total.loc[RFM_total['monetary_value']>0]   \
                        .loc[RFM_total['recency']>0]          \
                        .loc[RFM_total['frequency']>0]        \

In [ ]:
RFM_total.shape

In [ ]:
# df_forRFM[df_forRFM['Customer_ID']==12346.0].sort_values(by=['InvoiceDate'],ascending=False)
#df_forRFM[df_forRFM['Customer_ID']==12346.0].InvoiceDate.unique()
# len(df[df['Customer_ID']==12346.0].InvoiceDate.unique())
df_forRFM[df_forRFM['Customer_ID']==12347.0].InvoiceDate.unique()

In [ ]:
RFM_total.columns

In [ ]:
tit='Num repeat purchases (tot - 1)'
RFM_total['frequency'].plot(kind='hist',logy=True,bins=50,title=tit)

In [ ]:
tit='time bwn initial purchase and most recent (last) purchase'
RFM_total['recency'].plot(kind='hist',logy=True,bins=50,title=tit)

In [ ]:
RFM_total.plot.scatter(x='frequency',y='recency')

In [ ]:
pd.plotting.scatter_matrix(RFM_total, alpha=0.2,figsize=(15,15))

# Assumptions of the BG/NBD model:
- A customer’s relationship has two phases: “alive” for an unobserved period of time, then “dead”
- While alive, the number of transactions made by a customer follows a Poisson distribution with transaction rate lambda
- Heterogeneity in lambda follows a gamma distribution
- After any transaction, a customer dies with probability p; the probability that a customer dies after a number of transactions follows a geometric distribution
- p follows a beta distribution
- Lambda and p vary independently across customers

# seen these references
- https://medium.com/data-shopify/how-shopify-merchants-can-measure-retention-c12284bfed6f
- http://brucehardie.com/papers/bgnbd_2004-04-20.pdf (of which I have other url's as well)


# For follow up

- how many customers are there with more than 50/75 purchases ?
  Perhaps worth selecting those out and treating them separately ?

In [ ]:
help(BetaGeoFitter)

In [ ]:
beta_play = BetaGeoFitter(penalizer_coef=0.0)
beta_play.fit(RFM_total['frequency'], RFM_total['recency'], RFM_total['T']);

In [ ]:
# https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/plotting.py#L211
fig = plt.figure(figsize=(10,10))
plot_frequency_recency_matrix(beta_play);

In [ ]:
fig = plt.figure(figsize=(10,10))

plot_probability_alive_matrix(beta_play);

# compare for _number of transactions_ :
- prediction from the model, over period used for its fit
- the actual data, over the period used for its fit
## NOTE: this is a game done for the whole dataset, it's to learn how to play

In [ ]:
mf  = 100
for ly in [True,False]:
    # https://github.com/CamDavidsonPilon/lifetimes/blob/fb047f7471f07ca4416ddffdfeeca898404cda79/lifetimes/plotting.py#L25
    plot_period_transactions(beta_play, max_frequency=mf
                        ,title="Frequency of Repeat Transactions (total dataset)"
                        ,xlabel="Number of Calibration Period Transactions"
                        ,ylabel="Customers"
                        ,logy=ly
                        ,figsize=(30,10)    )

# SEPARATE data up-to June 2011 (calibration),
# from the whole dataset up to december (total)

In [ ]:
data_star='2009-12-01'
calib_end='2011-06-30'
obser_end='2011-12-09'

from datetime import date
d_data_star=date( * map(lambda u:int(u), data_star.split('-') ) )
d_calib_end=date( * map(lambda u:int(u), calib_end.split('-') ) )
d_obser_end=date( * map(lambda u:int(u), obser_end.split('-') ) )

data_span = (d_obser_end - d_data_star).days
data_train= (d_calib_end - d_data_star).days
print('Data cover %d days overall, of which %d are used for training'%(data_span,data_train))

In [ ]:
# https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/utils.py#L27

#  Create a summary of each customer over a calibration and test period.    
summary_cal_test = calibration_and_holdout_data(df_forRFM,'Customer_ID','InvoiceDate',
                                                    calibration_period_end=calib_end,
                                                    observation_period_end=obser_end)

summary_cal_test.shape

# XCHECK THIS SELECTION

In [ ]:
#summary_cal_test = summary_cal_test.loc[summary_cal_test['frequency_cal']>0]
#summary_cal_test.shape

In [ ]:
summary_cal_test.sort_values(by=['frequency_cal'],ascending=False).head(5)

# Fit the Beta-Geometric/NBD model, restricted up to June 2011
- restrict data up to the first semester of 2011
- fit the Beta-Geometric/NBD over that period
- use the remaining 6 months to qualify the fit

In [ ]:
beta_NDB_calib = BetaGeoFitter(penalizer_coef=0.0)

# The model is fit only using data up to the end of June 2011
beta_NDB_calib.fit(summary_cal_test['frequency_cal'] \
            ,summary_cal_test['recency_cal'] \
            ,summary_cal_test['T_cal'])

# Validation of the Beta-Geometric/NBD model
calibrated on the first segment of data, and evaluated on the last 6 months

# compare for _number of transactions_ :
- prediction from the model, over period used for its fit (Calib only, up to June 2011)
- the actual data, over the period used for its fit

In [ ]:
mf  = 100
for ly in [True,False]:
    # https://github.com/CamDavidsonPilon/lifetimes/blob/fb047f7471f07ca4416ddffdfeeca898404cda79/lifetimes/plotting.py#L25
    plot_period_transactions(beta_NDB_calib, max_frequency=mf
                        ,title="Num Transactions (calib dataset)"
                        ,xlabel="Number Transactions"
                        ,ylabel="Num Customers"
                        ,logy=ly
                        ,figsize=(10,6)    )

In [ ]:
fig = plt.figure(figsize=(9,6))
plot_cumulative_transactions(beta_NDB_calib, df_forRFM\
                             ,'InvoiceDate', 'Customer_ID'
                             ,data_span, data_train
                             ,title="Cumulative Transactions (up to June 2111)"
                             ,xlabel="day"
                             ,ylabel="Cumulative Transactions",)

In [ ]:
# is it correct we have only 25k transaction, as a result of aggregation of all of those into a single day?

In [ ]:
# calls plot_cumulative_transactions
# which in turns uses
#    expected_number_of_purchases_up_to_time()
#    method from the fitted model
#    to predict the cumulative number of purchases.

fig = plt.figure(figsize=(12,6))
plot_incremental_transactions(beta_NDB_calib, df_forRFM
                             ,'InvoiceDate', 'Customer_ID'
                             ,data_span, data_train
                             ,freq='D'
                             ,title="Num Transactions per day"
                             ,xlabel="day (since data start)"
                             ,ylabel="Transactions/day",)

In [ ]:
# The function

#             plot_calibration_purchases_vs_holdout_purchases
#             https://github.com/CamDavidsonPilon/lifetimes/blob/fb047f7471f07ca4416ddffdfeeca898404cda79/lifetimes/plotting.py#L77

# uses internally

#             conditional_expected_number_of_purchases_up_to_time
#             help(conditional_expected_number_of_purchases_up_to_time)
#             https://github.com/CamDavidsonPilon/lifetimes/blob/3e93e805ccb9b4660bd2609a7a9491237029547e/lifetimes/fitters/beta_geo_beta_binom_fitter.py#L179
# from the paper:
# for the expected number of transactions in a future period of length t for an individual with past observed behavior 

# to predict into the TEST period using the model,
# and plots the average purchase frequency in bins of other variables --<'kind'


## the plots have on ABCISSA bins of variables computed in the CALIBRATION period:

- k="frequency_cal"               # Purchases in calibration period,
- k="recency_cal"                 # Age of customer at last purchase,
- k="T_cal"                       # Age of customer at the end of calibration period,
- k="time_since_last_purchase"    # Time since user made last purchase

## the plots have on ordinate the average number of purchases over the TEST period:
- from the real data (the truth for this test)
- from the prediction


In [ ]:
ymax=8000
for k,xlab in [['frequency_cal','Num purchases (calib.)'],
            ['recency_cal','Recency i.e. t_last-t_first (calib.)'],
            ['T_cal','Age i.e. t_end_data-t_first (calib.)'],
            ['time_since_last_purchase','Tieme since last purchase (calib.)'],]:
                plot_calibration_purchases_vs_holdout_purchases(beta_NDB_calib
                                                    ,summary_cal_test
                                                    ,n=ymax # huge value => always all points 
                                                    ,figsize=(12,6)
                                                    ,title="Num Transactions per day"
                                                    ,kind=k )
                plt.title('Purchases after June 2011: test data vs prediction')
                plt.xlabel(xlab)
                plt.ylabel('Average number of purchases')


In [ ]:
summary_cal_test.head()

In [ ]:
RFM_total.head()

In [ ]:
# the following cell can be removed once I'll have put the 
# prob_alive somewhere else

In [ ]:
# select only the customers who have come back already at least once
# RFM_returned = RFM_total[RFM_total['frequency']>0]
summary_cal_test['prob_alive'] = \
 beta_NDB_calib.conditional_probability_alive(summary_cal_test['frequency_cal'],
                                               summary_cal_test['recency_cal'],
                                               summary_cal_test['T_cal'])
sns.distplot(summary_cal_test['prob_alive'])

# Fit the Beta-Geometric/NBD model, FULL Dataset                  (up to end December 2011)
- consider the whole dataset
- fit the Beta-Geometric/NBD over that period
- there''s no test perdiod for this model; we assume that the good accuracy observed when fitting over the limited period supports the validity of the model also over the whole pariod

In [ ]:
# Create a summary of each customer over the whole calibration period
# to implement that, set the end of the calibration period 3 days before the end of dataset
calib_end='2011-12-07'

summary_full_period = calibration_and_holdout_data(df_forRFM,'Customer_ID','InvoiceDate',
                                                    calibration_period_end=calib_end,
                                                    observation_period_end=obser_end)

# The model is fit trougouht the whole dataet
beta_NDB_total = BetaGeoFitter(penalizer_coef=0.0)

beta_NDB_total.fit(summary_full_period['frequency_cal'] \
            ,summary_full_period['recency_cal'] \
            ,summary_full_period['T_cal'])

RFM_total['prob_alive_total'] = \
       beta_NDB_total.conditional_probability_alive(RFM_total['frequency'],
                                        RFM_total['recency'],
                                        RFM_total['T'])
sns.distplot(RFM_total['prob_alive_total'])

In [ ]:
RFM_total.describe()

In [ ]:
summary_full_period.describe()

In [ ]:
tit='Alive probability at end of June 2011 (only calib dataset)'
summary_cal_test['prob_alive'].plot(kind='hist',bins=20,title=tit)

In [ ]:
tit='Alive probability at the end of 2011 (full dataset)'
RFM_total['prob_alive_total'].plot(kind='hist',bins=20,title=tit)

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.distplot(RFM_total['prob_alive_total'], hist_kws={'log':True})

In [ ]:
# merge dataframes such that prob_alive_total and prob_alive are available in the same df

In [ ]:
summary_cal_test.head(1)

In [ ]:
RFM_total.head(1)

In [ ]:
summary_final=summary_cal_test.join(RFM_total[['prob_alive_total','frequency','recency','T']])

In [ ]:
type(summary_final)

In [ ]:
summary_final.describe()

In [ ]:
summary_final.plot.scatter(x='prob_alive'
                           ,y='prob_alive_total'
                           ,figsize=(8,8)
                           ,c='DarkBlue')

In [ ]:
import matplotlib.colors as colors

fig = plt.figure(figsize=(10,8))

plt.hist2d(summary_final['prob_alive']
           , summary_final['prob_alive_total']
           , bins=30
           #, cmap='Blues'
           , cmap='plasma'
           , norm=colors.LogNorm()
          )
plt.xlabel('prob_alive (up to June 2011)')
plt.ylabel('prob_aliv_total')

cb = plt.colorbar()
cb.set_label('Num Customers')

In [ ]:
# SPOT CHECKS to make sense of the result...
# two customers were dead and 
# have come back to life shortly before the end of 2011, 
# see recency \approx== T, while T_cal very large
summary_final.loc[ (summary_final['prob_alive_total'] > 0.95) & \
                  (summary_final['prob_alive'] < 0.07) ]

In [ ]:
dead_up_threshold = 0.10
risk_up_threshold = 0.20

In [ ]:
# Answer to question number 1
summary_final.loc[ (summary_final['prob_alive_total'] < dead_up_threshold) & \
                   (summary_final['prob_alive']       > dead_up_threshold) ].describe()

In [ ]:
# Answer to question number 2
summary_final.loc[ (summary_final['prob_alive_total'] > dead_up_threshold) & \
                   (summary_final['prob_alive_total'] < risk_up_threshold) ].describe()

## Have any users churned during the second half of 2011? 
- I define them as users who have a low prob of being alive at the end of 2011 and had higher prob mid 2011
## Are there any users at high risk of churning by the end of 2011?
- users who have low live probabiliy at the end of 2011 and didn't mid 2011 

- split the users in three populations: dead, alive, high rish
- answer to first  question: number of users who were alive|risk in may 2011 and are dead at the end of 2011
- answer to second question: number of users who are at high risk at the end of 2011

## no need of splitting the customers to train/apply: I just train and verify goodness on variables other than those used. THERE IS NO GROUND TRUTH available in the training data


In [ ]:
# RFM_returned['prob_alive'].plot(kind='hist',logy=True,bins=20,title=tit)

In [ ]:
# sns.distplot(RFM_returned.loc[RFM_returned.prob_alive < 0.3]['prob_alive'])

In [ ]:
# RFM_returned.columns

In [ ]:
# prob_alive_sel = 0.1
# RFM_returned.loc[RFM_returned.prob_alive < prob_alive_sel].plot.scatter(x='prob_alive',y='monetary_value')
# the customers at risk of churning are not very very valuable
# print('there are %d customers with very low probability of being alive (i.e. %s or less)'% (len(RFM_returned.loc[RFM_returned.prob_alive < prob_alive_sel]),prob_alive_sel))

In [ ]:
# pd.plotting.scatter_matrix(RFM_returned, alpha=0.2,figsize=(15,15))